In [ ]:
import pandas as pd

# Assuming your file is named 'data.txt' and has '--' as the separator
file_path = '/content/FINAL.txt'

# Read the text file into a DataFrame
df = pd.read_csv(file_path, sep='--', header=None, names=['Question', 'Answer'])

# Display the DataFrame
print(df)

                                              Question  \
0                                             QUESTION   
1    What is KPR Institute of Engineering and Techn...   
2                             Where is KPRIET located?   
3           How many faculty members does KPRIET have?   
4          What is the strength of students at KPRIET?   
..                                                 ...   
182  What is the aim of Energy Sciences and Enginee...   
183  What is the focus of the Centre for IoT and AI...   
184  What is the objective of the Machining and Mat...   
185  What does the Nanoscience and Technology cente...   
186  What role do robots play in the Robotics and A...   

                                                Answer  
0                                               ANSWER  
1    KPR Institute of Engineering and Technology is...  
2    KPRIET is located in the suburbs of Coimbatore...  
3              KPRIET has 260 eminent faculty members.  
4                 

<ipython-input-21-9da756dad577>:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, sep='--', header=None, names=['Question', 'Answer'])


In [ ]:
df.head()

,Question,Answer
0,QUESTION,ANSWER
1,What is KPR Institute of Engineering and Techn...,KPR Institute of Engineering and Technology is...
2,Where is KPRIET located?,KPRIET is located in the suburbs of Coimbatore...
3,How many faculty members does KPRIET have?,KPRIET has 260 eminent faculty members.
4,What is the strength of students at KPRIET?,"KPRIET has 3,500 students."


In [ ]:
# Drop the first row from the DataFrame
df = df.drop(0)

# Reset the index after dropping the row
df = df.reset_index(drop=True)


In [ ]:
df.head()

,Question,Answer
0,What is KPR Institute of Engineering and Techn...,KPR Institute of Engineering and Technology is...
1,Where is KPRIET located?,KPRIET is located in the suburbs of Coimbatore...
2,How many faculty members does KPRIET have?,KPRIET has 260 eminent faculty members.
3,What is the strength of students at KPRIET?,"KPRIET has 3,500 students."
4,How many academic departments does KPRIET have?,KPRIET has 12 academic departments.


In [ ]:
df.iloc[30]

Question         Where did Kothai G complete her M.E. degree?
Answer      Kothai G completed her M.E. degree in Computer...
Name: 30, dtype: object

In [ ]:
df['Question'][3]

'What is the strength of students at KPRIET?'

# Building the chatbot

In [ ]:
!pip install "transformers==4.35" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" "tiktoken"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.9 MB/s eta 0:00:00
  Using cached nvidia_cu

In [ ]:
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from random import randrange
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer

## Dataset

In [ ]:
# Concatenate 'Question' and 'Answer' columns into a new column 'text'
df['text'] = 'Question:\n' + df['Question'] + '\n\nAnswer:\n' + df['Answer']

In [ ]:
print(df['text'].loc[1])

Question:
Where is KPRIET located?

Answer:
KPRIET is located in the suburbs of Coimbatore, a metropolitan city of Tamil Nadu,a southern Indian state.


In [ ]:
#drop columns other than 'text'
df.drop(columns=['Question','Answer'], axis=1, inplace=True)

In [ ]:
#convert to Huggingface Datasets format
train = Dataset.from_pandas(df)

In [ ]:
train

Dataset({
    features: ['text'],
    num_rows: 186
})

## Fine-Tuning

In [ ]:
model_id = "genaitraining/llama-2-7b-domain-tuned"

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-cr

In [ ]:
# Get the type
compute_dtype = getattr(torch, "float16")

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)


In [ ]:
%%time
# Load the pretrained model
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 24.6 s, sys: 31.1 s, total: 55.7 s
Wall time: 5min 8s


In [ ]:
%%time
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

CPU times: user 303 ms, sys: 42.2 ms, total: 345 ms
Wall time: 1.01 s


In [ ]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
                          lora_alpha=16,
                          lora_dropout=0.1,
                          r=64,
                          bias="none",
                          task_type="CAUSAL_LM"
                        )

In [ ]:
# Define the training arguments. For full list of arguments, check
#https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments
args = TrainingArguments(
    output_dir='llama2-7b-tuned-qna',
    num_train_epochs=10, # adjust based on the data size
    per_device_train_batch_size=2,
    save_strategy="epoch", #steps
    # evaluation_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    seed=42
)

In [ ]:
# Create the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    # eval_dataset=test,
    dataset_text_field='text',
    peft_config=peft_config,
    max_seq_length=1042,
    tokenizer=tokenizer,
    args=args,
    packing=True,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [ ]:
# train
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning

Step,Training Loss


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning

TrainOutput(global_step=60, training_loss=0.8680089314778646, metrics={'train_runtime': 435.592, 'train_samples_per_second': 4.27, 'train_steps_per_second': 2.135, 'total_flos': 4982267351531520.0, 'train_loss': 0.8680089314778646, 'epoch': 9.06})

In [ ]:
# save model in local
trainer.save_model()

## Merge the base model and adapters and save it

In [ ]:
# Empty VRAM
del model
del trainer
import gc
gc.collect()
gc.collect()

20730

In [ ]:
torch.cuda.empty_cache()

In [ ]:
gc.collect()

0

In [ ]:
%%time
from peft import AutoPeftModelForCausalLM

new_model = AutoPeftModelForCausalLM.from_pretrained(
    'llama2-7b-tuned-qna',
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 19.5 s, sys: 7.93 s, total: 27.4 s
Wall time: 1min 20s


## Test the model

In [ ]:
prompt = "What is kpr institute of engineering and technology?"

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

In [ ]:
outputs = new_model.generate(input_ids=input_ids,
                         max_new_tokens=200,
                        do_sample=True,
                        top_p=0.9,
                         temperature=0.2)

In [ ]:
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

In [ ]:
print(result)

What is kpr institute of engineering and technology?
Љ�?

KPR Institute of Engineering and Technology is a premier institute of higher learning in South India, established in 2009.
